In [1]:
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import copy
from mne.channels import find_ch_adjacency
%matplotlib qt5

# Funciones  análisis de clústeres

In [2]:
def matrices_cluster(grupo):
    # Diccionario con evocados de todos los sujetos, para ensayos congruentes e incongruentes
    congruentes = copy.deepcopy(evocados_co)
    incongruentes = copy.deepcopy(evocados_in)
    tiempos = congruentes['22100'].times
    adjacency, canales = find_ch_adjacency(congruentes['22100'].info, "eeg")

    # Matrices de evocados (tiempos, canales) de sujetos condición 1
    grupo_co = {}
    grupo_in = {}
    for sujeto in grupo:
        grupo_co[sujeto] = congruentes[sujeto]
        grupo_in[sujeto] = incongruentes[sujeto]

    ga_grupo_co = mne.grand_average(list(grupo_co.values()), interpolate_bads=True, drop_bads=True)
    ga_grupo_in = mne.grand_average(list(grupo_in.values()), interpolate_bads=True, drop_bads=True)

    co_grupo = [grupo_co[i].data for i in grupo_co]
    co_grupo = np.reshape(co_grupo,(-1,len(canales), len(tiempos)))
    co_grupo = np.transpose(co_grupo, axes=(0,2,1))
    print(co_grupo.shape)

    in_grupo = [grupo_in[i].data for i in grupo_in]
    in_grupo = np.reshape(in_grupo,(-1,len(canales), len(tiempos)))
    in_grupo = np.transpose(in_grupo, axes=(0,2,1))
    print(in_grupo.shape)

    return (ga_grupo_co, ga_grupo_in, co_grupo, in_grupo, tiempos, canales, adjacency)

In [3]:
# Evocados de clústeres
def ROI_evoked(evocado, canales):
    lista_canales = evocado.ch_names
    lista_indices = []
    for canal in canales:
        indice = lista_canales.index(canal)
        lista_indices.append(indice)
    ROI_evocado = mne.channels.combine_channels(evocado, groups={'ROI':lista_indices})
    return ROI_evocado

# Cálculo de media de clúster
def medidas(evoked, tmin, tmax):
    evoked_ROI = evoked.copy()
    evoked_ROI.crop(tmin = tmin, tmax = tmax)
    data = evoked_ROI.to_data_frame(time_format = None)
    data.set_index('time',inplace=True)
    mean_ROI = data.mean()
    return mean_ROI

In [4]:
# Diccionario con evocados de todos los sujetos, para ensayos congruentes e incongruentes
rutas_archivos = []
ruta_dir = os.getcwd() + '\\Sin filtrar\\'
dir = os.listdir(ruta_dir)
for q in range(len(dir)):
    ruta_arc =ruta_dir + dir[q]
    rutas_archivos.append(ruta_arc)

evocados_co = {}
evocados_in = {}

for l in range(len(rutas_archivos)):
    ruta=rutas_archivos[l]
    epochs = mne.read_epochs(ruta)
    epochs.filter(l_freq=None, h_freq=30)
    evoked_co = epochs['12','13','14','15','16','17','18','19'].average()
    evocados_co[ruta[-14:-9]] = evoked_co
    evoked_in = epochs['24','25','26','27','28','29','30','31'].average()
    evocados_in[ruta[-14:-9]] = evoked_in

Reading C:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\Sin filtrar\21100n_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     796.88 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
145 matching events found
No baseline correction applied
0 projection items activated
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 113 samples (0.441 sec)

Reading C:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\Sin filtrar\21101n_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     796.88 ms
        0 CTF compensation matrices available

In [0]:
evocados_co.keys()

In [2]:
# Gráfica de densidad
import seaborn as sns
# Configuro parámetros de gráficos
params = {'axes.labelsize':20, 'axes.titlesize': 20, 'axes.grid':True, 'axes.grid.axis':'both', 'axes.grid.which':'both','legend.fontsize':20, 'legend.loc': 'best', 'legend.title_fontsize':20, 'xtick.labelsize': 20, 'xtick.minor.visible': True, 'ytick.labelsize': 20, 'ytick.minor.visible': True}
plt.rcParams.update(params)
muestra = pd.read_csv('muestra_aleatoria_sin_ambos.csv')
muestra.rename(columns={'dscore_4':'IAT score', 'prejudice_level':'IAT effect level group'}, inplace=True)
density_plot = sns.kdeplot(data=muestra, x='IAT score', hue='IAT effect level group', fill=True, alpha=0.3)
density_plot.set_alpha=0.3

In [6]:
# Matrices de muestra de sujetos a analizar
muestra = pd.read_csv('muestra_aleatoria_sin_ambos.csv')
negative = list(muestra[muestra['prejudice_level']=='negative']['subject'])
neutral = list(muestra[muestra['prejudice_level']=='neutral']['subject'])
positive = list(muestra[muestra['prejudice_level']=='positive']['subject'])
print(negative)

[23009, 22111, 23013, 21122, 22100, 22116, 22110, 21113, 22113, 21140, 23002, 21139, 23016, 23017, 23006, 21118, 22107, 21123, 22108]


In [7]:
for i in range(len(negative)):
    negative[i] = str(negative[i])
for i in range(len(neutral)):
    neutral[i] = str(neutral[i])
for i in range(len(positive)):
    positive[i] = str(positive[i])
__, __, negative_co, negative_in, __, __, __ = matrices_cluster(negative)
__, __, neutral_co, neutral_in, __, __, __ = matrices_cluster(neutral)
__, __, positive_co, positive_in, tiempos, canales, adjacency = matrices_cluster(positive)

# Matrices de diferencias entre ensayos
negative_dif = negative_co - negative_in
neutral_dif = neutral_co - neutral_in
positive_dif = positive_co - positive_in
print(np.shape(positive_dif))

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(19, 256, 64)
(19, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(37, 256, 64)
(37, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(29, 256, 64)
(29, 256, 64)
(29, 256, 64)


# Medidas de clústeres y gráficas de ERPs

In [13]:
def lista_evocados_grupos(canales, tmin=0.0, tmax=0.796875):
    negative_co = {}
    negative_in = {}
    negative_dif = {}
    for sujeto in negative:
        negative_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        negative_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = negative_co[sujeto].data - negative_in[sujeto].data
        negative_dif[sujeto] = mne.EvokedArray(data=data_dif, info=negative_in[sujeto].info, tmin=negative_in[sujeto].tmin)
    neutral_co = {}
    neutral_in = {}
    neutral_dif = {}
    for sujeto in neutral:
        neutral_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        neutral_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = neutral_co[sujeto].data - neutral_in[sujeto].data
        neutral_dif[sujeto] = mne.EvokedArray(data=data_dif, info=neutral_in[sujeto].info, tmin=neutral_in[sujeto].tmin)
    positive_co = {}
    positive_in = {}
    positive_dif = {}
    for sujeto in positive:
        positive_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        positive_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = positive_co[sujeto].data - positive_in[sujeto].data
        positive_dif[sujeto] = mne.EvokedArray(data=data_dif, info=positive_in[sujeto].info, tmin=positive_in[sujeto].tmin)

    return negative_co, neutral_co, positive_co, negative_in, neutral_in, positive_in,negative_dif,neutral_dif,positive_dif

In [14]:
def lista_evocados_grupos_2(canales, tmin=0.0, tmax=0.796875):
    groups = {'negative':negative, 'neutral':neutral, 'positive':positive}
    results = {}
    for group in groups:
        co, in_, dif = {}, {}, {}
        for sujeto in groups[group]:
            co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
            in_[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
            data_dif = co[sujeto].data - in_[sujeto].data
            dif[sujeto] = mne.EvokedArray(data=data_dif, info=in_[sujeto].info, tmin=in_[sujeto].tmin)
        results[group+'_co'] = co
        results[group+'_in'] = in_
        results[group+'_dif'] = dif
    return results

In [15]:
# Configuro parámetros de gráficos
params = {'axes.labelsize':20, 'axes.titlesize': 20, 'axes.grid':True, 'axes.grid.axis':'both', 'axes.grid.which':'both','legend.fontsize':20, 'legend.loc': 'lower left','xtick.labelsize': 20, 'xtick.minor.visible': True, 'ytick.labelsize': 20, 'ytick.minor.visible': True}
plt.rcParams.update(params)

In [16]:
canales = ['FZ', 'FC1', 'FC2']
tmin=0.0
tmax=0.796875
negative_co,neutral_co,positive_co,negative_in,neutral_in,positive_in,negative_dif,neutral_dif,positive_dif = lista_evocados_grupos(canales=canales, tmin=tmin, tmax=tmax)
negative_co

{'23009': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '22111': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '23013': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '21122': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '22100': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '22116': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '22110': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '21113': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '22113': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '21140': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '23002': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '21139': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, 

In [17]:
# Diccionarios de toda la muestra
ROI_co = {**negative_co,**neutral_co,**positive_co}
ROI_in = {**negative_in,**neutral_in,**positive_in}
ROI_dif = {**negative_dif,**neutral_dif,**positive_dif}

In [60]:
def erps_grupos(canales, tmin=0.0, tmax=0.796875):
    # Diccionarios de evocados de clústeres por grupos
    negative_co,neutral_co,positive_co,negative_in,neutral_in,positive_in,negative_dif,neutral_dif,positive_dif = lista_evocados_grupos(canales=canales, tmin=tmin, tmax=tmax)

    # Diccionarios de toda la muestra
    ROI_co = {**negative_co,**neutral_co,**positive_co}
    ROI_in = {**negative_in,**neutral_in,**positive_in}
    ROI_dif = {**negative_dif,**neutral_dif,**positive_dif}

    # Listas de evocados de clústeres por grupos
    negative_co_list = list(negative_co.values())
    neutral_co_list = list(neutral_co.values())
    positive_co_list = list(positive_co.values())
    negative_in_list = list(negative_in.values())
    neutral_in_list = list(neutral_in.values())
    positive_in_list = list(positive_in.values())
    negative_dif_list = list(negative_dif.values())
    neutral_dif_list = list(neutral_dif.values())
    positive_dif_list = list(positive_dif.values())

    erps = {'negative_co':negative_co_list, 'neutral_co':neutral_co_list, 'positive_co':positive_co_list, 'negative_in':negative_in_list, 'neutral_in':neutral_in_list, 'positive_in':positive_in_list}
    erps_dif = {'negative_dif':negative_dif_list, 'neutral_dif':neutral_dif_list, 'positive_dif':positive_dif_list}
    return  erps, erps_dif

In [61]:
canales = ['FZ', 'FC1', 'FC2']
erps, erps_dif = erps_grupos(canales=canales)

In [73]:
fig1 = mne.viz.plot_compare_evokeds(evokeds=erps, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff', '#d62728ff', '#9467bdff', '#8c564bff'], linestyles=['solid', 'dashdot', 'dashed', 'solid', 'dashdot', 'dashed'], axes=None, ci=None, truncate_yaxis=False, truncate_xaxis=False, show_sensors=False, legend='lower left', split_legend=False, combine=None, show=False)
ax = fig1[0].gca()
rect1 = plt.Rectangle((0.14,-1.8), 0.09, 3.6, color='lightcyan')
ax.add_patch(rect1)
rect2 = plt.Rectangle((0.29,-1.8), 0.08, 3.6, color='lightcyan')
ax.add_patch(rect2)
rect3 = plt.Rectangle((0.46,-1.8), 0.06, 3.6, color='lightcyan')
ax.add_patch(rect3)
rect4 = plt.Rectangle((0.54,-1.8), 0.26, 3.6, color='lightcyan')
ax.add_patch(rect4)

In [76]:
fig2 = mne.viz.plot_compare_evokeds(evokeds=erps_dif, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff'], linestyles=['solid', 'dashdot', 'dashed'],axes=None, ci=None, truncate_yaxis=False, truncate_xaxis=False, show_sensors=False, legend='lower left', split_legend=False, combine=None, show=False)
ax = fig2[0].gca()
rect1 = plt.Rectangle((0.14,-1.8), 0.09, 3.6, color='lightcyan')
ax.add_patch(rect1)
rect2 = plt.Rectangle((0.29,-1.8), 0.08, 3.6, color='lightcyan')
ax.add_patch(rect2)
rect3 = plt.Rectangle((0.46,-1.8), 0.06, 3.6, color='lightcyan')
ax.add_patch(rect3)
rect4 = plt.Rectangle((0.54,-1.8), 0.26, 3.6, color='lightcyan')
ax.add_patch(rect4)